In [2]:

from hcve_lib.utils import notebook_init

notebook_init()

from mlflow import set_tracking_uri

%autoreload 2
from deps.common import get_variables_cached
from pandas import DataFrame
import plotly.figure_factory as ff

set_tracking_uri('http://localhost:5000')

data, metadata, X, y = get_variables_cached()

- Using euclidian distance didn't make so much send in the end.

In [28]:
from deps.pipelines import get_preprocessing
from hcve_lib.wrapped_sklearn import DFPipeline, DFStandardScaler
from sklearn.metrics import euclidean_distances

preprocessing = DFPipeline([
    ('scaler', DFStandardScaler()),
    *get_preprocessing(X)[0],
])

X_transformed = preprocessing.fit_transform(X, y)
X_groups = X_transformed.groupby(data['STUDY'])

distance = euclidean_distances(X_groups.get_group('ASCOT'), X_groups.get_group('FLEMENGHO'))

In [38]:
distance_matrix = {
    cohort1_name: {
        cohort2_name: euclidean_distances(cohort1, cohort2).mean()
        for cohort2_name, cohort2 in  X_groups
    }
    for cohort1_name, cohort1 in  X_groups
}

In [44]:
def plot_matrix(matrix: DataFrame) -> None:
    matrix = matrix.iloc[::-1]
    fig = ff.create_annotated_heatmap(
        matrix.round(2).to_numpy(),
        x=list(matrix.columns),
        y=list(matrix.index),
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

plot_matrix(DataFrame(distance_matrix))